### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import timeit
start_time = timeit.default_timer() #Useful at the end for total execution time

In [2]:
#DISACTIVATING SCIENTIFIC NOTATION
pd.options.display.float_format = '{:.2f}'.format

In [3]:
#pd.set_option('display.max_rows', None)

Importing metaData dataset as trips and explainatory analysis

In [4]:
csv_file_path = 'trips.csv'
selected_columns = ['name', 'busNumber', 'drivenDistance', 'busRoute', 'energyConsumption',
                     'itcs_numberOfPassengers_mean', 'status_gridIsAvailable_mean', 'temperature_ambient_mean']
# Define data types for selected columns
dtype_dict1 = {
    'busNumber': np.int32,
    'drivenDistance': np.float32,
    'energyConsumption': np.float32,
    'itcs_numberOfPassengers_mean': np.float32,
    'status_gridIsAvailable_mean': np.float32,
    'temperature_ambient_mean': np.float32,
    'name': str,
    'busRoute': str
}
trips = pd.read_csv(csv_file_path, usecols=selected_columns,dtype=dtype_dict1)

In [5]:
trips.dtypes

name                             object
busNumber                         int32
drivenDistance                  float32
busRoute                         object
energyConsumption               float32
itcs_numberOfPassengers_mean    float32
status_gridIsAvailable_mean     float32
temperature_ambient_mean        float32
dtype: object

In [6]:
#replace '-' with NA
trips.replace('-', np.nan, inplace=True)

In [7]:
trips.dtypes

name                             object
busNumber                         int32
drivenDistance                  float32
busRoute                         object
energyConsumption               float32
itcs_numberOfPassengers_mean    float32
status_gridIsAvailable_mean     float32
temperature_ambient_mean        float32
dtype: object

In [8]:
trips.isnull().sum() #See if there are Null values in the dataset

name                             0
busNumber                        0
drivenDistance                   0
busRoute                        11
energyConsumption                0
itcs_numberOfPassengers_mean     0
status_gridIsAvailable_mean      0
temperature_ambient_mean         0
dtype: int64

#### Data Cleaning 

In [9]:
# Convert temperature to celsius 
trips["temperature_ambient_mean"] = trips["temperature_ambient_mean"] - 273.15

In [10]:
trips.head()

,name,busNumber,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,status_gridIsAvailable_mean,temperature_ambient_mean
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00,5.54,0.74,9.23
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,59029.60,31,402258496.00,33.11,0.86,14.39
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,240900.41,33,1445732992.00,19.69,0.78,15.60
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,42565.48,NaN,281986688.00,1.69,0.77,9.26
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,125277.20,72,620725824.00,23.75,0.91,11.58


#### Importing ZTBus dataset  as all_trips_details and data manipulation

###### After evaluating the dimension of the dataset and the difficulty to operate on such a large amount of data i have decided to import only relevant columns that are necessary for the assignment questions.By doing that i managed to operate faster on the data. I also have 32Gb of RAM and a very fast CPU so computation time is fast enought to operate on the data with max 5 minutes of loading and preparing data (acceptable time for me). 

In [17]:
# glob is useful in order to get all the file names from a folder
# ATTENTION! i have unzipped the folder before operating on it 
import glob 
files = glob.glob('C:/Users/rusuf/OneDrive - Università degli Studi di Milano-Bicocca/PRIMO ANNO/Python Project/details dataset/*.csv')  # Update the path to your file directory /*

In [19]:
# Columns to exclude (these columns are not relevant for the project questions)
columns_to_exclude = ['itcs_busRoute', 'odometry_articulationAngle', 'odometry_steeringAngle',
                       'traction_tractionForce', 'odometry_wheelSpeed_fl', 'odometry_wheelSpeed_fr',
                       'odometry_wheelSpeed_ml', 'odometry_wheelSpeed_mr', 'odometry_wheelSpeed_rl',
                       'odometry_wheelSpeed_rr', 'status_doorIsOpen', 'status_gridIsAvailable', 'traction_brakePressure',
                       'time_unix','gnss_course','gnss_latitude','gnss_longitude']

# Define data types for selected columns using NumPy
dtype_dict = {
    'electric_powerDemand': np.float32,
    'gnss_altitude': np.float32,
    'itcs_numberOfPassengers': np.float32,
    'odometry_vehicleSpeed': np.float32,
    'status_haltBrakeIsActive': np.float32,
    'status_parkBrakeIsActive': np.float32,
    'temperature_ambient': np.float32,
}

# Concatenate all DataFrames directly excluding specified columns and applying dtype conversion
all_trip_details = pd.concat(
    (pd.read_csv(file, usecols=lambda col: col not in columns_to_exclude, dtype=dtype_dict)
     .assign(name=file.split('\\')[-1].split('.')[0])
     for file in files),
    ignore_index=True
)

In [49]:
all_trip_details.dtypes

time_iso                    datetime64[ns, UTC]
electric_powerDemand                    float32
gnss_altitude                           float32
itcs_numberOfPassengers                 float32
itcs_stopName                            object
odometry_vehicleSpeed                   float32
status_haltBrakeIsActive                   bool
status_parkBrakeIsActive                   bool
temperature_ambient                     float64
name                                     object
altitude_variation                      float32
month                                     int32
dtype: object

In [21]:
# Replace the '-' with NAs (NOT USING IT)
#all_trip_details.replace('-', np.nan, inplace=True)

In [22]:
# Convert 'time_iso' to datetime
all_trip_details['time_iso'] = pd.to_datetime(all_trip_details['time_iso'], errors='coerce')

In [23]:
# Convert 'temperature_ambient' to celsius from Kelvin
all_trip_details['temperature_ambient'] = all_trip_details['temperature_ambient'] - 273.15

In [24]:
# Converting to BOOL
all_trip_details['status_haltBrakeIsActive'] = all_trip_details['status_haltBrakeIsActive'].astype(bool)
all_trip_details['status_parkBrakeIsActive'] = pd.to_numeric(all_trip_details['status_parkBrakeIsActive'], errors='coerce').astype(bool)

In [25]:
# May be useful (NOT IN USE)
#all_trip_details = all_trip_details.dropna(how='any', inplace=True)

In [26]:
all_trip_details.head()

,time_iso,electric_powerDemand,gnss_altitude,itcs_numberOfPassengers,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name
0,2019-04-30 03:18:56+00:00,-13.85,NaN,NaN,-,0.00,False,False,20.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
1,2019-04-30 03:18:57+00:00,-3.85,NaN,NaN,-,0.00,False,False,19.22,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
2,2019-04-30 03:18:58+00:00,-0.67,NaN,NaN,-,0.00,False,False,19.78,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
3,2019-04-30 03:18:59+00:00,-1.09,NaN,NaN,-,0.00,False,False,20.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20
4,2019-04-30 03:19:00+00:00,-0.81,NaN,NaN,-,0.00,False,False,20.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20


In [27]:
# Select only specific columns from 'trips' dataset
selected_columns = ['name', 'busNumber', 'drivenDistance', 'busRoute', 'energyConsumption']
trips_selected = trips[selected_columns]

# Merge 'all_trip_details' with the selected columns from 'trips' based on 'name'
merged_df = pd.merge(all_trip_details, trips_selected, on='name', how='left')

merged_df.head(10)

,time_iso,electric_powerDemand,gnss_altitude,itcs_numberOfPassengers,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name,busNumber,drivenDistance,busRoute,energyConsumption
0,2019-04-30 03:18:56+00:00,-13.85,NaN,NaN,-,0.00,False,False,20.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
1,2019-04-30 03:18:57+00:00,-3.85,NaN,NaN,-,0.00,False,False,19.22,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
2,2019-04-30 03:18:58+00:00,-0.67,NaN,NaN,-,0.00,False,False,19.78,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
3,2019-04-30 03:18:59+00:00,-1.09,NaN,NaN,-,0.00,False,False,20.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
4,2019-04-30 03:19:00+00:00,-0.81,NaN,NaN,-,0.00,False,False,20.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
5,2019-04-30 03:19:01+00:00,-0.37,NaN,NaN,-,0.00,False,False,20.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
6,2019-04-30 03:19:02+00:00,-0.72,NaN,NaN,-,0.00,False,False,20.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
7,2019-04-30 03:19:03+00:00,-1.63,NaN,NaN,-,0.00,False,False,19.64,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
8,2019-04-30 03:19:04+00:00,-1.32,NaN,NaN,-,0.00,False,False,19.18,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
9,2019-04-30 03:19:05+00:00,-0.51,NaN,NaN,-,0.00,False,False,19.62,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00


In [28]:
merged_df.isnull().sum()

time_iso                           0
electric_powerDemand               0
gnss_altitude                1330884
itcs_numberOfPassengers     48229724
itcs_stopName                      0
odometry_vehicleSpeed              0
status_haltBrakeIsActive           0
status_parkBrakeIsActive           0
temperature_ambient              103
name                               0
busNumber                          0
drivenDistance                     0
busRoute                      545269
energyConsumption                  0
dtype: int64

In [29]:
merged_df.dtypes

time_iso                    datetime64[ns, UTC]
electric_powerDemand                    float32
gnss_altitude                           float32
itcs_numberOfPassengers                 float32
itcs_stopName                            object
odometry_vehicleSpeed                   float32
status_haltBrakeIsActive                   bool
status_parkBrakeIsActive                   bool
temperature_ambient                     float64
name                                     object
busNumber                                 int32
drivenDistance                          float32
busRoute                                 object
energyConsumption                       float32
dtype: object

In [30]:
merged_df.head()

,time_iso,electric_powerDemand,gnss_altitude,itcs_numberOfPassengers,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name,busNumber,drivenDistance,busRoute,energyConsumption
0,2019-04-30 03:18:56+00:00,-13.85,NaN,NaN,-,0.00,False,False,20.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
1,2019-04-30 03:18:57+00:00,-3.85,NaN,NaN,-,0.00,False,False,19.22,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
2,2019-04-30 03:18:58+00:00,-0.67,NaN,NaN,-,0.00,False,False,19.78,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
3,2019-04-30 03:18:59+00:00,-1.09,NaN,NaN,-,0.00,False,False,20.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00
4,2019-04-30 03:19:00+00:00,-0.81,NaN,NaN,-,0.00,False,False,20.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00


## Solving Ex.

1) Extract all trips with busRoute 83

In [31]:
trips_bus_83 = trips[trips["busRoute"]=='83']
trips_bus_83.head()

,name,busNumber,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,status_gridIsAvailable_mean,temperature_ambient_mean
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,225047.91,83,1544278016.00,23.48,0.47,7.40
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,224512.30,83,1631815936.00,17.42,0.45,6.74
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,77824.36,83,540601280.00,23.18,0.46,7.90
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,225095.80,83,1692171008.00,20.96,0.48,6.69
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,224181.20,83,1145859968.00,17.21,0.34,14.19


2) Extract all trips where busRoute is not a number

In [32]:
# Count all the null routes 
trips.isnull().sum()

name                             0
busNumber                        0
drivenDistance                   0
busRoute                        11
energyConsumption                0
itcs_numberOfPassengers_mean     0
status_gridIsAvailable_mean      0
temperature_ambient_mean         0
dtype: int64

In [33]:
any_null_routes = trips[trips['busRoute'].isnull()]
any_null_routes

,name,busNumber,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,status_gridIsAvailable_mean,temperature_ambient_mean
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,77213.87,NaN,478585216.00,5.54,0.74,9.23
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,42565.48,NaN,281986688.00,1.69,0.77,9.26
9,B183_2019-05-10_03-16-11_2019-05-10_18-51-37,183,210577.00,NaN,1303390976.00,8.23,0.74,14.41
10,B183_2019-05-13_03-10-23_2019-05-13_23-16-13,183,267033.81,NaN,1647431936.00,7.89,0.80,11.53
19,B183_2019-05-24_02-52-47_2019-05-24_22-35-11,183,263432.59,NaN,1448056960.00,7.52,0.76,19.99
25,B183_2019-06-01_05-05-41_2019-06-01_15-15-16,183,132710.70,NaN,837011776.00,6.71,0.86,23.55
26,B183_2019-06-01_15-19-38_2019-06-01_23-12-03,183,104331.10,NaN,506560512.00,6.40,0.73,24.18
34,B183_2019-06-21_02-54-48_2019-06-21_22-37-37,183,256945.59,NaN,1603927040.00,7.91,0.77,21.42
36,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,183,205184.91,NaN,1462189056.00,9.74,0.78,26.86
37,B183_2019-06-25_03-22-22_2019-06-25_22-24-45,183,243475.20,NaN,1733085056.00,9.94,0.80,28.65


3) For each (busNumber, busRoute) pair, determine the number of trips

In [34]:
total_trips = trips.groupby(["busNumber","busRoute"])["name"].size()
# Ricorda trips non numeric 
total_trips

busNumber  busRoute
183        31           12
           32           12
           33          130
           46          104
           72          114
           83          441
           N1           10
           N2           19
           N4           11
208        31            5
           32           14
           33           25
           46           19
           72           44
           83          405
           N1            6
           N2           20
           N4            7
Name: name, dtype: int64

4) For each trip, compute the ratio between the energy consumption and the average number of passengers

In [35]:
trip_ratios = trips.groupby(['name']).apply(lambda x: x['energyConsumption'] / x['itcs_numberOfPassengers_mean'])
trip_ratios

name                                              
B183_2019-04-30_03-18-56_2019-04-30_08-44-20  0       86405008.00
B183_2019-04-30_13-22-07_2019-04-30_17-54-02  1       12147475.00
B183_2019-05-01_05-58-51_2019-05-01_22-32-30  2       73427936.00
B183_2019-05-03_02-50-21_2019-05-03_05-53-20  3      167332784.00
B183_2019-05-03_15-41-57_2019-05-03_23-06-24  4       26131896.00
                                                         ...     
B208_2022-12-06_14-43-49_2022-12-06_18-22-52  1404    10702153.00
B208_2022-12-07_05-13-02_2022-12-07_19-19-53  1405    53384612.00
B208_2022-12-08_05-22-20_2022-12-08_18-39-15  1406    47380468.00
B208_2022-12-09_23-55-12_2022-12-10_03-24-28  1407    22477526.00
B208_2022-12-10_23-34-46_2022-12-11_03-29-05  1408    33457694.00
Length: 1409, dtype: float32


5. For each station (itcs_stopName), determine the average number of passengers.

In [36]:
average_passengers_by_station = merged_df.groupby('itcs_stopName')['itcs_numberOfPassengers'].mean()
average_passengers_by_station

itcs_stopName
-                                  NaN
Zürich, Albisrank                16.20
Zürich, Albisriederplatz         25.45
Zürich, Altes Krematorium        21.98
Zürich, Bahnhof Affoltern         2.85
                                  ... 
Zürich, Zentrum Witikon          17.26
Zürich, Zweiackerstrasse          7.50
Zürich, Zwielplatz                2.05
Zürich, Zwinglihaus              23.20
Zürich,Kalkbreite/Bhf.Wiedikon   23.16
Name: itcs_numberOfPassengers, Length: 150, dtype: float32

6. For each station, determine the buses that have stopped there at least once

In [37]:
station_counts = merged_df.groupby(['itcs_stopName', 'busNumber']).size().reset_index(name='stop_count') #i reset the index just for aestethical reason

stations_with_at_least_10_stops = station_counts[station_counts['stop_count'] > 1]

stations_with_at_least_10_stops

,itcs_stopName,busNumber,stop_count
0,-,183,30598244
1,-,208,17631480
2,"Zürich, Albisrank",183,6324
3,"Zürich, Albisrank",208,6199
4,"Zürich, Albisriederplatz",183,11218
...,...,...,...
294,"Zürich, Zwielplatz",208,4
295,"Zürich, Zwinglihaus",183,2791
296,"Zürich, Zwinglihaus",208,1231
297,"Zürich,Kalkbreite/Bhf.Wiedikon",183,279


7. For each station, determine the buses that have stopped there at least ten times.

In [38]:
station_counts = merged_df.groupby(['itcs_stopName', 'busNumber']).size().reset_index(name='stop_count')

stations_with_at_least_10_stops = station_counts[station_counts['stop_count'] >= 10]

stations_with_at_least_10_stops

,itcs_stopName,busNumber,stop_count
0,-,183,30598244
1,-,208,17631480
2,"Zürich, Albisrank",183,6324
3,"Zürich, Albisrank",208,6199
4,"Zürich, Albisriederplatz",183,11218
...,...,...,...
293,"Zürich, Zwielplatz",183,77
295,"Zürich, Zwinglihaus",183,2791
296,"Zürich, Zwinglihaus",208,1231
297,"Zürich,Kalkbreite/Bhf.Wiedikon",183,279


9. For each (route, bus) pair, compute the ratio between the overall energy consumption and the overall driven distance.

In [39]:
#We have to sum energy consuption for each (route,bus) and divide it by the sum of the driven distance (total trips)
ratio = merged_df.groupby(['busRoute','busNumber']).apply(lambda x: x['energyConsumption'].sum() / x['drivenDistance'].sum())
ratio

busRoute  busNumber
31        183         5946.82
          208         5126.06
32        183         6237.65
          208         5455.06
33        183         5969.75
          208         5665.92
46        183         5620.39
          208         5572.63
72        183         5888.67
          208         5418.46
83        183         5835.24
          208         5862.87
N1        183         5997.31
          208         5614.01
N2        183         5718.81
          208         5419.19
N4        183         6180.96
          208         6046.23
dtype: float32

10. Starting from the results of the previous point, for each route compute the buses with max and min energy ratio, and save the difference between these ratios in a dataframe.

In [40]:
ratio_df = ratio.reset_index(name='energy_ratio')

# Find the buses with max and min energy ratio for each route
max_energy_bus = ratio_df.loc[ratio_df.groupby('busRoute')['energy_ratio'].idxmax()]
min_energy_bus = ratio_df.loc[ratio_df.groupby('busRoute')['energy_ratio'].idxmin()]

# Merge DataFrames to get the difference
difference_df = pd.merge(max_energy_bus, min_energy_bus, on='busRoute', suffixes=('_max', '_min'))

# Compute the difference between max and min energy ratios
difference_df['difference'] = difference_df['energy_ratio_max'] - difference_df['energy_ratio_min']

difference_df

,busRoute,busNumber_max,energy_ratio_max,busNumber_min,energy_ratio_min,difference
0,31,183,5946.82,208,5126.06,820.76
1,32,183,6237.65,208,5455.06,782.58
2,33,183,5969.75,208,5665.92,303.84
3,46,183,5620.39,208,5572.63,47.76
4,72,183,5888.67,208,5418.46,470.22
5,83,208,5862.87,183,5835.24,27.63
6,N1,183,5997.31,208,5614.01,383.30
7,N2,183,5718.81,208,5419.19,299.63
8,N4,183,6180.96,208,6046.23,134.73



11. Find the bus maximizing the difference computed in the previous point.

In [41]:
max_difference_bus = difference_df.loc[difference_df['difference'].idxmax()]
print("Bus maximizing the difference:")
max_difference_bus

Bus maximizing the difference:


busRoute                31
busNumber_max          183
energy_ratio_max   5946.82
busNumber_min          208
energy_ratio_min   5126.06
difference          820.76
Name: 0, dtype: object

12. Extract the rows of the details such that the gnss_altitude differs from the value in the preceding row. Store also the difference in the variable altitude_variation.

In [42]:
# Create a new column 'altitude_variation' to store the difference between consecutive altitudes
all_trip_details['altitude_variation'] = all_trip_details['gnss_altitude'].diff()

# Extract rows where 'altitude_variation' is not equal to 0 and not null
filtered_rows = all_trip_details[(all_trip_details['altitude_variation'] != 0) & ~all_trip_details['altitude_variation'].isnull()]

filtered_rows.head(10)


,time_iso,electric_powerDemand,gnss_altitude,itcs_numberOfPassengers,itcs_stopName,odometry_vehicleSpeed,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,name,altitude_variation
148,2019-04-30 03:21:24+00:00,32607.16,397.83,NaN,-,3.79,False,False,19.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,0.03
149,2019-04-30 03:21:25+00:00,39229.61,400.31,NaN,-,4.05,False,False,19.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,2.48
150,2019-04-30 03:21:26+00:00,54537.21,401.69,NaN,-,4.40,False,False,19.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,1.38
151,2019-04-30 03:21:27+00:00,52785.54,400.49,NaN,-,4.86,False,False,19.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,-1.20
152,2019-04-30 03:21:28+00:00,105212.20,399.12,NaN,-,5.39,False,False,19.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,-1.37
153,2019-04-30 03:21:29+00:00,89766.56,400.71,NaN,-,6.07,False,False,19.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,1.59
154,2019-04-30 03:21:30+00:00,91947.33,401.20,NaN,-,6.61,False,False,19.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,0.50
155,2019-04-30 03:21:31+00:00,37132.31,401.50,NaN,-,6.94,False,False,19.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,0.30
156,2019-04-30 03:21:32+00:00,5441.70,401.52,NaN,-,6.90,False,False,19.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,0.02
157,2019-04-30 03:21:33+00:00,5715.44,403.10,NaN,-,6.80,False,False,19.00,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,1.59


13. For each details dataset, compute the sum of the absolute value (i.e. the sign is not considered) of altitude_variation.


In [43]:
sum_abs_altitude_variation = all_trip_details.groupby('name')['altitude_variation'].apply(lambda x: x.abs().sum())
sum_abs_altitude_variation

name
B183_2019-04-30_03-18-56_2019-04-30_08-44-20     184.74
B183_2019-04-30_13-22-07_2019-04-30_17-54-02    3896.83
B183_2019-05-01_05-58-51_2019-05-01_22-32-30     637.41
B183_2019-05-03_02-50-21_2019-05-03_05-53-20    4074.48
B183_2019-05-03_15-41-57_2019-05-03_23-06-24   13325.95
                                                 ...   
B208_2022-12-06_14-43-49_2022-12-06_18-22-52    2300.40
B208_2022-12-07_05-13-02_2022-12-07_19-19-53    8617.68
B208_2022-12-08_05-22-20_2022-12-08_18-39-15    8022.73
B208_2022-12-09_23-55-12_2022-12-10_03-24-28    2748.42
B208_2022-12-10_23-34-46_2022-12-11_03-29-05    3247.22
Name: altitude_variation, Length: 1409, dtype: float32

14. For each month of the year, compute the average ambient temperature

In [44]:
all_trip_details['month'] = all_trip_details['time_iso'].dt.month
# Compute the average ambient temperature for each month
average_temp_by_month = all_trip_details.groupby('month')['temperature_ambient'].mean()
average_temp_by_month

month
1     5.65
2     8.06
3    10.08
4    13.89
5    17.03
6    23.62
7    24.57
8    23.18
9    19.29
10   14.74
11    8.75
12    6.41
Name: temperature_ambient, dtype: float64


15. For each bus compute the total time when the halt brake is active and the total time when the park brake is active. Compute also the ratio between those two times.

In [45]:
# Set 'time_iso' as the DataFrame index
merged_df.set_index('time_iso', inplace=True)

# Compute total time when the halt brake is active for each bus
halt_brake_time = merged_df.groupby(['busNumber'])['status_haltBrakeIsActive'].sum()

# Compute total time when the park brake is active for each bus
park_brake_time = merged_df.groupby(['busNumber'])['status_parkBrakeIsActive'].sum()

# Compute the ratio 
ratio = halt_brake_time / park_brake_time

# Remember that the time is in seconds
print("Total Time with Halt Brake:")
print(halt_brake_time)
print("\nTotal Time with Park Brake:")
print(park_brake_time)
print("\nRatio between Halt Brake and Park Brake Times:")
print(ratio)

Total Time with Halt Brake:
busNumber
183    7837579
208    5942201
Name: status_haltBrakeIsActive, dtype: int64

Total Time with Park Brake:
busNumber
183    3184686
208    1828738
Name: status_parkBrakeIsActive, dtype: int64

Ratio between Halt Brake and Park Brake Times:
busNumber
183   2.46
208   3.25
dtype: float64


16. For each pair of stops that are consecutive --in at least a trip--, compute the average speed achieved when going from the first to the second stop.

In [46]:
# Sort the DataFrame by trip name and time_iso to ensure consecutive stops are adjacent
all_trips_details = all_trip_details.sort_values(by=['name', 'time_iso'])

# Replace '-' with NaN in 'itcs_stopName' 
all_trips_details['itcs_stopName'] = all_trips_details['itcs_stopName'].replace('-', np.nan)

# Choose a specific trip name
selected_trip = 'B183_2019-04-30_03-18-56_2019-04-30_08-44-20'

# Filter DataFrame for the selected trip and create a copy
selected_trip_df = all_trips_details[all_trips_details['name'] == selected_trip].copy()

# Use .loc to set values on the copy of the DataFrame
selected_trip_df.loc[:, 'trip_segment'] = selected_trip_df['itcs_stopName'].ffill()

# Group by 'trip_segment' and calculate the mean speed including NA, AKA '-' stops
avg_speed_selected_trip = selected_trip_df.groupby('trip_segment')['odometry_vehicleSpeed'].mean().reset_index()

# Create a new column 'STOPS' combining the first and last stop in each segment
avg_speed_selected_trip['STOPS'] = avg_speed_selected_trip['trip_segment'].shift(1) + ' - ' + avg_speed_selected_trip['trip_segment']

# Display the resulting DataFrame with the average speed between consecutive stops for the selected trip
result_selected_trip = avg_speed_selected_trip[['STOPS', 'odometry_vehicleSpeed']]
result_selected_trip.columns = ['STOPS', 'Average Speed']
result_selected_trip = result_selected_trip.dropna().reset_index(drop=True)
print(result_selected_trip)


                                                STOPS  Average Speed
0   Zürich, Bahnhof Tiefenbrunnen - Zürich, Bahnho...           5.42
1         Zürich, Bahnhofplatz/HB - Zürich, Bethanien           5.30
2      Zürich, Bethanien - Zürich, Botanischer Garten           4.56
3   Zürich, Botanischer Garten - Zürich, Bäckeranlage           7.03
4              Zürich, Bäckeranlage - Zürich, Central          10.38
5              Zürich, Central - Zürich, Freiestrasse           3.35
6      Zürich, Freiestrasse - Zürich, Fröhlichstrasse           5.33
7      Zürich, Fröhlichstrasse - Zürich, Güterbahnhof           4.16
8            Zürich, Güterbahnhof - Zürich, Hardplatz           5.43
9           Zürich, Hardplatz - Zürich, Hegibachplatz           4.41
10     Zürich, Hegibachplatz - Zürich, Herdernstrasse           6.71
11  Zürich, Herdernstrasse - Zürich, Hinterbergstr...           3.00
12     Zürich, Hinterbergstrasse - Zürich, Hofstrasse           2.77
13        Zürich, Hofstrasse - Zür

In [47]:
#TOTAL EXECUTION TIME (circa 5 minutes on average with 32gb RAM and Ryzen 7 PRO 5850U GPU)

end_time = timeit.default_timer()
execution_time = end_time - start_time

print(f"Execution time: {execution_time} seconds")

Execution time: 708.5908171000192 seconds


In [1]:

#-----------------------------------------------------------------------------------------
#WARNING READ BEFORE EXECUTION 
# This is an extension that allows you to compute the average speed between consecutive stops in all the trips 
#-----------------------------------------------------------------------------------------

# Sort the DataFrame by trip name and time_iso to ensure consecutive stops are adjacent
all_trips_details = all_trip_details.sort_values(by=['name', 'time_iso'])

# Replace '-' with NaN in 'itcs_stopName' to facilitate further processing
all_trips_details['itcs_stopName'] = all_trips_details['itcs_stopName'].replace('-', np.nan)

# Create a new column 'trip_segment' to index consecutive stops for each trip
all_trips_details['trip_segment'] = all_trips_details.groupby(['name'])['itcs_stopName'].ffill()

# Group by 'name' and 'trip_segment', and calculate the mean speed including NA, AKA '-' stops
avg_speed_df = all_trips_details.groupby(['name', 'trip_segment'])['odometry_vehicleSpeed'].mean().reset_index()

# Create a new column 'STOPS' combining the first and last stop in each segment
avg_speed_df['STOPS'] = avg_speed_df.groupby(['name'])['trip_segment'].transform(lambda x: f"{x.iloc[0]} - {x.iloc[-1]}")

# Display the resulting DataFrame with the average speed between consecutive stops
result_df = avg_speed_df[['STOPS', 'odometry_vehicleSpeed']] 
result_df.columns = ['STOPS', 'Average Speed']
result_df = result_df.drop_duplicates().reset_index(drop=True)
result_df.groupby('STOPS')['Average Speed'].mean() 


NameError: name 'all_trip_details' is not defined